# Statistical analysis (Single Population) on BuildingPermits_all_filled

Univariate analysis is perhaps the simplest form of statistical analysis. The key fact is that only one variable is involved.

Bivariate analysis involves the analysis of two variables (often denoted as X, Y), for the purpose of determining the empirical relationship between them.

<center>**For bivariate analysis please use the Statistical analysis (Multiple Populations) variant of this notebook.**</center>

* [Setup and loading the data](#setup)
* [Preprocessing of the data](#preprocessing)
* [Statistical analysis and vizualisation](#general)
* [Single population tests](#tests_single)

<center><strong>Select Cell > Run All to execute the whole analysis</strong></center>

## Setup and dataset loading <a id="setup" /> 

First of all, let's load the libraries that we'll use

In [0]:
%pylab inline

In [0]:
import dataiku                               # Access to Dataiku datasets
import pandas as pd, numpy as np             # Data manipulation 
from matplotlib import pyplot as plt         # Graphing
import seaborn as sns                        # Graphing
#sns.set(style="white")                       # Tuning the style of charts
import warnings                              # Disable some warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from scipy import stats                      # Stats

The first thing we do is now to load the dataset and put aside the three main types of columns:

* Numerics
* Categorical
* Dates

Statistical analysis requires having the data in memory, we are only going to load a sample of the data. Modify the following cell to change the size of the sample.

In [0]:
dataset_limit = 10000

Load a DSS dataset as a Pandas dataframe

In [0]:
# Take a handle on the dataset
mydataset = dataiku.Dataset("BuildingPermits_all_filled")

# Load the first lines.
# You can also load random samples, limit yourself to some columns, or only load
# data matching some filters.
#
# Please refer to the Dataiku Python API documentation for more information
df = mydataset.get_dataframe(limit = dataset_limit)

df_orig = df.copy()

# Get the column names
numerical_columns = list(df.select_dtypes(include=[np.number]).columns)
categorical_columns = list(df.select_dtypes(include=[object]).columns)
date_columns = list(df.select_dtypes(include=['<M8[ns]']).columns)

# Print a quick summary of what we just loaded
print "Loaded dataset"
print "   Rows: %s" % df.shape[0]
print "   Columns: %s (%s num, %s cat, %s date)" % (df.shape[1], 
                                                    len(numerical_columns), len(categorical_columns),
                                                    len(date_columns))

## Preprocessing of the data <a id="preprocessing" />

We will assume that the values are in the first numerical column.

In [0]:
value_col = numerical_columns[0]

Uncomment the following lines to take control on this

In [0]:
#value_col = u'my_value_column'
print "Selected value column is '%s'" % (value_col)

We impute missing values in the value column

In [0]:
# Use mean for numerical features
v = df[value_col].mean()
if np.isnan(v):
    v = 0
print "Filling value column '%s' with %s" % (value_col, v)
df[value_col] = df[value_col].fillna(v)
df_pop_1 = df[value_col]

## Statistical analysis and vizualisation <a id="general" /a>

### General statistics
Number of records, mean, standard deviation, minimal value, quartiles, maximum value, mode, variance, skewness and kurtosis.

In [0]:
additional_stats = ["var", "skew", "kurtosis"]
print "Stats about your series:\n", df_pop_1.describe().append(pd.Series(NaN if df_pop_1.mode().empty else df_pop_1.mode()[0], index=["mode"])).append(pd.Series([df_pop_1.__getattr__(x)() for x in additional_stats], index=additional_stats))

### Histogram
Histograms let you see the number of occurrences in your value column.

In [0]:
plt.title("Histogram of "+value_col);
plt.hist(df_pop_1);

### Distplot
Distplots combine an histogram with a kernel density estimation.

In [0]:
sns.distplot(df_pop_1);

### Box plot

A simple way of representing statistical data on a plot in which a rectangle is drawn to represent the second and third quartiles, with a vertical line inside to indicate the median value. The lower and upper quartiles are shown as horizontal lines either side of the rectangle.

In [0]:
sns.boxplot(df_pop_1);

### Violin plot

The violin plot is similar to box plots, except that they also show the probability density of the data at different values. Violin plots include a marker for the median of the data and a box indicating the interquartile range, as in standard box plots. Overlaid on this box plot is a kernel density estimation. 

In [0]:
sns.violinplot(df_pop_1);

### Letter value plot

Letter value plots are an improvement upon boxplots for large datasets.

They display the median and the quartiles, like a standard box plot, but will also draw boxes for subsequent "eights", "sixteenth" etc... which are generically called letter values.

A cut off condition will leave a reasonable number of outliers out of the final boxes, helping you spot them easily.

Letter value plots give a good sense of the distribution of data, and of its skewness.

In [0]:
sns.lvplot(df_pop_1);

## Statistical testing <a id="tests" /a>

**Reminder:** For a given significance level (e.g. 0.05), if the resulting p-value is smaller (p < 0.05), the null hypothesis is rejected. Otherwise (p ≥ 0.05) it cannot be rejected.

In [0]:
# Define your confidence threshold here, default is 0.05
confidence = 0.05

In [0]:
def analyse_results(confidence, pvalue, message, population_name="your series"):
    if pvalue < confidence:
        print "The hypothesis of " + message + " for "+ population_name + " is rejected with pvalue %s (smaller than %s)" % (pvalue, confidence)
    else:
        print "The hypothesis of " + message + " for "+ population_name + " can not be rejected, pvalue was %s (greater than %s)" % (pvalue, confidence)

## Single population tests <a id="tests_single" /a>

### Goodness of fit with a normal law: Shapiro-Wilk test

The null-hypothesis of this test is that the population is normally distributed.

In [0]:
pvalue_1 = stats.shapiro(df_pop_1)[1]
test = 'normal distribution'
analyse_results(confidence, pvalue_1, test)

### Test for the average value

The null-hypothesis of this test is that the population has the specified mean.

In [0]:
# Define the mean you ant to test for here
tested_mean = 0

In [0]:
pvalue_1 = stats.ttest_1samp(df_pop_1, tested_mean).pvalue
test = 'mean=%s' % (tested_mean)
analyse_results(confidence, pvalue_1, test)

<center>**If you have multiple populations and want to run bivariate analyses please use the Statistical analysis (Multiple Populations) variant of this notebook.**</center>